# CREA LA TABLA DE TRANSICIONES DEL 8 PUZZLE

In [ ]:
from collections import deque

# Define moves relative to the position of '9' (empty space)
directions = [-3, 1, 3, -1]  # Change in index for each move

# Validate if a move is within bounds
def is_valid_move(index, direction):
    if direction == -3 and index < 3:  # Up
        return False
    if direction == 3 and index > 5:  # Down
        return False
    if direction == -1 and index % 3 == 0:  # Left
        return False
    if direction == 1 and index % 3 == 2:  # Right
        return False
    return True

# Swap two positions in a state
def swap(state, i, j):
    state = list(state)
    state[i], state[j] = state[j], state[i]
    return ''.join(state)

# Generate transitions for the 8-puzzle
def generate_transitions_8_puzzle(initial_state):
    visited = set()
    queue = deque([initial_state])
    transitions = {}

    while queue:
        current_state = queue.popleft()
        if current_state in visited:
            continue
        visited.add(current_state)

        empty_index = current_state.index('9')
        state_transitions = []

        for direction in directions:
            if is_valid_move(empty_index, direction):
                new_state = swap(current_state, empty_index, empty_index + direction)
                state_transitions.append(new_state)
                if new_state not in visited:
                    queue.append(new_state)
            else:
                state_transitions.append(None)  # Invalid move

        transitions[current_state] = state_transitions

    return transitions

def invert_transitions(transitions):
    inverted = {}
    for state, next_states in transitions.items():
        for next_state in next_states:
            if next_state:
                if next_state not in inverted:
                    inverted[next_state] = []
                inverted[next_state].append(state)
    return inverted

def calculate_steps_to_end(transitions, goal_state):
    steps_to_end = {}
    queue = deque([(goal_state, 0)])  # (state, distance)
    visited = set()

    while queue:
        current_state, distance = queue.popleft()
        if current_state in visited:
            continue
        visited.add(current_state)
        steps_to_end[current_state] = distance - 1

        for prev_state in transitions.get(current_state, []):
            if prev_state not in visited:
                queue.append((prev_state, distance + 1))

    return steps_to_end

def calculate_path_lengths(transitions, initial_state):
    path_lengths = {}
    queue = deque([(initial_state, 0)])  # (state, path_length)
    visited = set()

    while queue:
        current_state, path_length = queue.popleft()
        if current_state in visited:
            continue
        visited.add(current_state)
        path_lengths[current_state] = path_length

        for next_state in transitions.get(current_state, []):
            if next_state and next_state not in visited:
                queue.append((next_state, path_length + 1))

    return path_lengths

def compare_steps_and_paths(steps_to_end, path_lengths):
    discrepancies = []
    for state, steps in steps_to_end.items():
        path_length = path_lengths.get(state, None)
        if path_length is not None and steps != path_length:
            discrepancies.append((state, steps, path_length))
    return discrepancies

# Initial state of the puzzle
initial_state = "123459678"
# Goal state of the puzzle (solved state)
goal_state = "123456789"

# Generate the transitions
T = generate_transitions_8_puzzle(initial_state)

# Invert the transitions for reverse lookup
inverted_T = invert_transitions(T)

# Calculate steps to end
steps_to_end = calculate_steps_to_end(inverted_T, goal_state)

# Calculate path lengths
path_lengths = calculate_path_lengths(T, initial_state)

# Compare steps_to_end and path_lengths
discrepancies = compare_steps_and_paths(steps_to_end, path_lengths)

# Print the resulting discrepancies
if discrepancies:
    print("Discrepancies found:")
    for state, steps, path_length in discrepancies:
        print(f"State: {state}, Steps to End: {steps}, Path Length: {path_length}")
else:
    print("No discrepancies found.")


Se han truncado las últimas 5000 líneas del flujo de salida.
State: 475639281, Steps to End: 26, Path Length: 22
State: 147938625, Steps to End: 26, Path Length: 24
State: 645981327, Steps to End: 26, Path Length: 22
State: 634581297, Steps to End: 26, Path Length: 22
State: 634951287, Steps to End: 26, Path Length: 22
State: 564931287, Steps to End: 26, Path Length: 22
State: 694231857, Steps to End: 26, Path Length: 20
State: 641237895, Steps to End: 26, Path Length: 20
State: 634915827, Steps to End: 26, Path Length: 22
State: 634987251, Steps to End: 26, Path Length: 24
State: 634857291, Steps to End: 26, Path Length: 24
State: 694837251, Steps to End: 26, Path Length: 24
State: 654819237, Steps to End: 26, Path Length: 24
State: 654831297, Steps to End: 26, Path Length: 24
State: 478965231, Steps to End: 26, Path Length: 28
State: 148357692, Steps to End: 26, Path Length: 22
State: 487953621, Steps to End: 26, Path Length: 22
State: 487659213, Steps to End: 26, Path Length: 22
Sta

Por ejemplo T["123456789"]=['123459786', None, None, '123456798']
 y T["123456789"][1]=None

#TABLA DE RECOMPENSAS
En lugar de usar la distancia de Manhattan directamente como recompensa, tomamos el valor máximo posible de la distancia (en este caso, el valor máximo para un estado es 12, que es la distancia máxima posible en el 8-puzzle) y restamos la distancia de Manhattan de este valor.

Esto asegurará que los estados más cercanos a la solución tengan una mayor recompensa.

In [ ]:
"""def manhattan_distance_reward(state):
    goal_state = "123456789"  # Solución del puzzle
    max_distance = 12  # Distancia máxima posible en el 8-puzzle (si las piezas están en las posiciones opuestas)

    distance = 0
    for i, char in enumerate(state):
        if char != '9':  # Ignorar el espacio vacío
            goal_index = goal_state.index(char)
            # Convertir índices 1D a coordenadas 2D (fila, columna)
            x1, y1 = divmod(i, 3)
            x2, y2 = divmod(goal_index, 3)
            # Sumar la distancia Manhattan
            distance += abs(x1 - x2) + abs(y1 - y2)
    # Recompensa inversa: cuanto más cerca de la solución, mayor la recompensa
    return max_distance - distance """

'def manhattan_distance_reward(state):\n    goal_state = "123456789"  # Solución del puzzle\n    max_distance = 12  # Distancia máxima posible en el 8-puzzle (si las piezas están en las posiciones opuestas)\n\n    distance = 0\n    for i, char in enumerate(state):\n        if char != \'9\':  # Ignorar el espacio vacío\n            goal_index = goal_state.index(char)\n            # Convertir índices 1D a coordenadas 2D (fila, columna)\n            x1, y1 = divmod(i, 3)\n            x2, y2 = divmod(goal_index, 3)\n            # Sumar la distancia Manhattan\n            distance += abs(x1 - x2) + abs(y1 - y2)\n    # Recompensa inversa: cuanto más cerca de la solución, mayor la recompensa\n    return max_distance - distance '

In [ ]:
def sequence_reward(state):
  goal_state = "123456789"  # Estado objetivo

  correct_sequence = ""  # Secuencia correcta acumulada

  # Construir la secuencia correcta desde el inicio del estado
  for i, char in enumerate(state):
    if char == goal_state[i]:
      correct_sequence += char
    else:
      break  # Detenerse en la primera pieza incorrecta

  # La recompensa será igual al número de piezas correctas en secuencia
  reward=1000
  if len(correct_sequence)<9:
    reward=(len(correct_sequence)-1)*2 # Recompensa base negativa
  return reward


In [ ]:
R = {}
# Asignar recompensas basadas en la secuencia correcta
for state in T.keys():
  R[state] = sequence_reward(state)

In [ ]:
R["123459678"]

8

## Descripción:
manhattan_distance_reward invierte la lógica de la recompensa, otorgando más puntos cuanto más cerca está un estado de la solución.
generate_reward_table_inverted genera la tabla de recompensas con la lógica corregida.
Con esta modificación, los estados más cercanos a la solución recibirán recompensas más altas, lo que es útil si se está entrenando un agente con un enfoque como Q-learning o cualquier otro algoritmo basado en recompensas. la recompensa del estado R['123456789'] será 500

In [ ]:
"""# Generar la tabla de recompensas con la recompensa inversa
def generate_reward_table_inverted(transitions):
    reward_table = {}
    for state in transitions.keys():
      recompensa=manhattan_distance_reward(state)
      if recompensa==12:
        recompensa=500
      reward_table[state] = recompensa
    return reward_table"""

'# Generar la tabla de recompensas con la recompensa inversa\ndef generate_reward_table_inverted(transitions):\n    reward_table = {}\n    for state in transitions.keys():\n      recompensa=manhattan_distance_reward(state)\n      if recompensa==12:\n        recompensa=500\n      reward_table[state] = recompensa\n    return reward_table'

In [ ]:
# Generar la tabla de recompensas invertida para el puzzle
"""R=generate_reward_table_inverted(T)"""

'R=generate_reward_table_inverted(T)'

## INICIALIZACIÓN de la tabla Q a cero

In [ ]:
# Función para inicializar la tabla Q a cero
def initialize_Q_table(transitions):
    Q_table = {}
    # Inicializar tabla Q a cero para cada estado y cada acción posible
    for state in transitions.keys():
        #Q_table[state] = {action: 0 for action in move_names}
        Q_table[state] = {action: 0 for action in range(4)}
    return Q_table #, df_Q

In [ ]:
Q=initialize_Q_table(T)

##Q-LEARNING
Ejemplo de Estado aleatorio seleccionado: 621849375
Transiciones válidas desde este estado: ['629841375', None, '621845379', '621894375'].
T['621849375'][2]='621845379'
Q['123456789'] vale inicialmente {0: 0, 1: 0, 2: 0, 3: 0}

## Q-learnning sin épsilon greedy

In [ ]:
#import numpy as np
#HIPERPARÁMETROS
#v = 0.1  # Factor de actualización (también conocido como tasa de aprendizaje)
#y = 0.9  # Factor de descuento
# SELECCIÓN DEL ESTADO INICIAL
# Listar todos los estados posibles
#estados = list(T.keys())
# Seleccionar un estado aleatorio de los posibles estados
#s = np.random.choice(estados) # ESTADO INICIAL
#entrenar = 0
#max_entrenamientos = 10000000  # Número de episodios de entrenamiento
#while entrenar < max_entrenamientos:
    # Acción aleatoria al azar (elige un número entre 0 y 3)
    #a = np.random.randint(4)
    # Asegurarse de que la transición es válida
    #while T[s][a] == None:
        #a = np.random.randint(4)  # Reintentar si la acción no es válida
    # Estado siguiente
    #siguiente = T[s][a]
    # Actualizar la tabla Q con la fórmula de Q-learning
    #Q[s][a] = (1 - v) * Q[s][a] + v * (R[s] + y * max(Q[siguiente].values())) # ECUACIÓN DE BELLMAN
    # Mostrar el estado y el siguiente estado
    #print(f"{s} --> {siguiente}")
    #if entrenar%1000000==0:
      #print(Q[s])
    # Actualizar el estado actual
    #if siguiente != "123456789":
        #s = siguiente  # Estado siguiente
    #else:
        # Si llegamos al estado 26, volvemos a un estado al azar
        #s=np.random.choice(list(T.keys()))
    # Incrementar el contador de episodios
    #entrenar += 1

## Q-learnning con épsilon greedy

In [ ]:
import numpy as np

# HIPERPARÁMETROS
v = 0.7  # Tasa de aprendizaje
y = 0.8  # Factor de descuento
epsilon = 0.95  # Probabilidad de exploración inicial
epsilon_min = 0.01  # Epsilon mínimo
epsilon_decay = 0.99999995  # Factor de decaimiento de epsilon

# SELECCIÓN DEL ESTADO INICIAL
estados = list(T.keys())
s = np.random.choice(estados)
entrenar = 0
max_entrenamientos = 10000000

while entrenar < max_entrenamientos:
    # Filtrar acciones válidas
    acciones_validas = [a for a in Q[s] if T[s][a] is not None]

    if not acciones_validas:
        print(f"No hay acciones válidas desde el estado {s}")
        break  # Si no hay acciones válidas, salir del ciclo

    # Seleccionar acción usando epsilon-greedy
    if np.random.rand() < epsilon:
        # Exploración: elegir una acción válida al azar
        a = np.random.choice(acciones_validas)
    else:
        # Explotación: elegir la mejor acción válida
        a = max(acciones_validas, key=lambda a: Q[s][a])

    # Obtener el siguiente estado
    siguiente = T[s][a]

    # Actualizar la tabla Q usando la ecuación de Bellman
    Q[s][a] = (1 - v) * Q[s][a] + v * (R[s] + y * max(Q[siguiente].values()))

    # Mostrar progreso cada 10,000 episodios
    if entrenar % 100000 == 0:
        print(f"Episodio {entrenar}, Epsilon: {epsilon:.4f}")
        print(Q[s])

    # Actualizar el estado actual
    if siguiente != "123456789":
        s = siguiente
    else:
        # Si llegamos al estado objetivo, reiniciar a un estado aleatorio
        s = np.random.choice(estados)

    # Reducir epsilon para priorizar explotación a medida que se aprende
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # Incrementar el contador de episodios
    entrenar += 1



Episodio 0, Epsilon: 0.9500
{0: 0, 1: 0, 2: 0, 3: 0.0}
Episodio 100000, Epsilon: 0.9453
{0: 0.0, 1: 0, 2: 0.0, 3: 0.0}
Episodio 200000, Epsilon: 0.9405
{0: 0, 1: 0, 2: 0, 3: -1.4}
Episodio 300000, Epsilon: 0.9359
{0: 0, 1: 0, 2: -1.4, 3: 0}
Episodio 400000, Epsilon: 0.9312
{0: -1.946, 1: 0, 2: -1.8199999999999998, 3: 0}
Episodio 500000, Epsilon: 0.9265
{0: 0, 1: -1.4, 2: 0, 3: -1.8199999999999998}
Episodio 600000, Epsilon: 0.9219
{0: -1.9995626, 1: 0, 2: -1.998542, 3: -2.9652000000000003}
Episodio 700000, Epsilon: 0.9173
{0: -1.4, 1: -1.946, 2: 0, 3: 0}
Episodio 800000, Epsilon: 0.9127
{0: 0, 1: 0.0, 2: 0, 3: 0.0}
Episodio 900000, Epsilon: 0.9082
{0: -2.6039999999999996, 1: -1.998542, 2: 0, 3: -1.9951400000000001}
Episodio 1000000, Epsilon: 0.9037
{0: -1.946, 1: 0, 2: -1.946, 3: -3.003}
Episodio 1100000, Epsilon: 0.8992
{0: -3.10947, 1: -1.9838, 2: 0, 3: -1.8199999999999998}
Episodio 1200000, Epsilon: 0.8947
{0: 0, 1: -1.4, 2: -1.8199999999999998, 3: 0}
Episodio 1300000, Epsilon: 0.890

## SOLUCIÓN

In [ ]:
import numpy as np
path = []

# Mostrar los pasos para resolver el 8-puzzle desde un estado inicial aleatorio
def resolver_8_puzzle(T, Q, steps_to_end, estado_inicial, estado_objetivo="123456789"):
    estado_actual = estado_inicial
    pasos = 0

    while estado_actual != estado_objetivo:
        acciones = Q[estado_actual]
        # Filtrar acciones válidas
        acciones_validas = [a for a in Q[estado_actual] if T[estado_actual][a] is not None]
        if not acciones_validas:
            break

        accion_mejor = max(acciones_validas, key=acciones.get)
        siguiente_estado = T[estado_actual][accion_mejor]

        if siguiente_estado is None:
            break

        estado_actual = siguiente_estado
        pasos += 1

        if pasos > 100:  # Limitar a 100 pasos para evitar bucles infinitos
            break

    # Calcular porcentaje de aprendizaje
    if estado_actual == estado_objetivo:
        distancia_ideal = steps_to_end.get(estado_inicial, None)
        if distancia_ideal is not None and pasos > 0:
            return (distancia_ideal / pasos) * 100
    return 0

# Simulación de episodios
def entrenar_y_evaluar(T, Q, steps_to_end, num_episodios=1000000, evaluacion_cada=100000):
    aprendizaje_porcentaje = []
    for episodio in range(1, num_episodios + 1):
        # Simulación de un episodio (aquí puedes entrenar `Q` según tu lógica)

        # Evaluar cada `evaluacion_cada` episodios
        if episodio % evaluacion_cada == 0:
            print(f"Evaluando en el episodio {episodio}...")
            estados_prueba = np.random.choice(list(T.keys()), size=10, replace=False)
            porcentajes = [
                resolver_8_puzzle(T, Q, steps_to_end, estado_inicial=estado)
                for estado in estados_prueba
            ]
            promedio_aprendizaje = np.mean(porcentajes)
            aprendizaje_porcentaje.append((episodio, promedio_aprendizaje))
            print(f"Porcentaje de aprendizaje promedio: {promedio_aprendizaje:.2f}%")

    return aprendizaje_porcentaje

# Ejemplo de uso
estado_inicial = None  # Deja como None para seleccionar un estado aleatorio
num_episodios = 1000000  # Total de episodios
evaluacion_cada = 100000  # Evaluar cada 100,000 episodios

# Asegúrate de tener `T`, `Q`, y `steps_to_end` definidos
resultados_aprendizaje = entrenar_y_evaluar(T, Q, steps_to_end, num_episodios, evaluacion_cada)


Evaluando en el episodio 100000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 200000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 300000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 400000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 500000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 600000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 700000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 800000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 900000...
Porcentaje de aprendizaje promedio: 0.00%
Evaluando en el episodio 1000000...
Porcentaje de aprendizaje promedio: 0.00%


Está entrenando pero el estado inicial no se encuentra en el campo de entrenamiento

In [ ]:
T

{'123459678': ['129453678', None, '123458679', '123495678'],
 '129453678': [None, None, '123459678', '192453678'],
 '123458679': ['123459678', None, None, '123458697'],
 '123495678': ['193425678', '123459678', '123475698', '123945678'],
 '192453678': [None, '129453678', '152493678', '912453678'],
 '123458697': ['123498657', '123458679', None, '123458967'],
 '193425678': [None, '139425678', '123495678', '913425678'],
 '123475698': ['123495678', '123475689', None, '123475968'],
 '123945678': ['923145678', '123495678', '123645978', None],
 '152493678': ['192453678', '152439678', '152473698', '152943678'],
 '912453678': [None, '192453678', '412953678', None],
 '123498657': ['193428657', '123489657', '123458697', '123948657'],
 '123458967': ['123958467', '123458697', None, None],
 '139425678': [None, None, '135429678', '193425678'],
 '913425678': [None, '193425678', '413925678', None],
 '123475689': ['123479685', None, None, '123475698'],
 '123475968': ['123975468', '123475698', None, None]

In [ ]:
Q

{'123459678': {0: 35.199999999999974,
  1: 0,
  2: 39.99999999999997,
  3: 38.399999999999984},
 '129453678': {0: 0, 1: 0, 2: 33.99999999999997, 3: 23.759999999999987},
 '123458679': {0: 39.99999999999997, 1: 0, 2: 0, 3: 39.99999999999997},
 '123495678': {0: 30.319999999999993,
  1: 37.99999999999998,
  2: 35.119999999999976,
  3: 33.51999999999999},
 '192453678': {0: 0,
  1: 27.19999999999998,
  2: 17.407999999999987,
  3: 15.80799999999999},
 '123458697': {0: 38.399999999999984,
  1: 39.99999999999997,
  2: 0,
  3: 39.99999999999997},
 '193425678': {0: 0,
  1: 19.45599999999999,
  2: 30.399999999999984,
  3: 17.85599999999999},
 '123475698': {0: 36.39999999999998,
  1: 34.09599999999998,
  2: 0,
  3: 34.09599999999998},
 '123945678': {0: 24.41599999999999,
  1: 34.399999999999984,
  2: 29.215999999999994,
  3: 0},
 '152493678': {0: 21.759999999999987,
  1: 13.926399999999989,
  2: 13.926399999999989,
  3: 13.926399999999989},
 '912453678': {0: 0, 1: 19.759999999999987, 2: 9.046399999

In [ ]:
Q["541872369"]


{0: -1.999999971302186, 1: 0, 2: 0, 3: -1.9999996811354}

In [ ]:
T["541872369"]

['541879362', None, None, '541872396']

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.pyplot as plt

def animate_puzzle_solution(path):
    """
    Muestra la animación del puzzle 8 desde el estado inicial al estado objetivo.

    path: Lista de estados que representan el camino desde el estado inicial al objetivo.
    """
    # Crear la figura
    fig, ax = plt.subplots()
    ax.axis("off")
    tiles = [[None for _ in range(3)] for _ in range(3)]

    # Inicialización del gráfico
    def init():
        for i in range(3):
            for j in range(3):
                tiles[i][j] = ax.text(
                    j, -i, "", ha="center", va="center", fontsize=20,
                    bbox=dict(boxstyle="round", facecolor="white", edgecolor="black")
                )
        ax.set_xlim(-0.5, 2.5)
        ax.set_ylim(-2.5, 0.5)
        return [tile for row in tiles for tile in row]

    # Actualización para cada frame
    def update(frame):
        state = path[frame]
        for i, value in enumerate(state):
            x, y = i % 3, i // 3
            tiles[y][x].set_text("" if value == '9' else value)
        return [tile for row in tiles for tile in row]

    # Crear la animación
    anim = FuncAnimation(fig, update, frames=len(path), init_func=init, blit=True, repeat=False)
    plt.close(fig)
    return HTML(anim.to_jshtml())

# ============================
# EJEMPLO DE USO
# ============================

# Simulación de un camino de solución (puedes usar tu función find_path_to_goal para generarlo)

# Mostrar la animación
animate_html = animate_puzzle_solution(path)
display(animate_html)

#Siguientes pasos:
- 1 SE DEBE CALCULAR LA DISTANCIA A LA SOLUCIÓN DESDE CADA ESTADO EXAMINANDO LA TABLA T
- 1.1 SE INICIALIZAN TODOS LOS ESTADOS CON DISTANCIA -1
- 1.2 SE MARCA EL ESTADO SOLUCIÓN CON DISTANCIA 0
- 1.3 SE MARCAN CON DISTANCIA 1 LOS ESTADOS CONTIGUOS A LA SOLUCIÓN
- 1.4 SE MARCAN CON DISTANCIA K LOS ESTADOS (NO MARCADOS) CONTIGUOS A ALGUNO DE DISTANCIA K-1 SE REPITE HASTA QUE NO SE MARQUE NINGUNO MÁS.

## Justificación de los pasos en el código:
- El código utiliza la tabla de transiciones T para examinar los estados vecinos a partir del estado objetivo. Esto se hace en la función calculate_steps_to_end

- Esto está implícito, ya que el diccionario steps_to_end se construye agregando estados a medida que se visitan, y cualquier estado que aún no esté en el diccionario tiene una distancia no definida (conceptualmente -1). Al marcar las distancias, el valor inicial sería tratado como -1.

- Se marca el estado solución con distancia 0
```
queue = deque([(goal_state, 0)])

- En la primera iteración del bucle, los estados vecinos al objetivo (distancia 0) son procesados y marcados con distancia 1:
```
for prev_state, next_states in transitions.items():
    if current_state in next_states and prev_state not in visited:
        queue.append((prev_state, distance + 1))

- Se marcan con distancia K los estados (no marcados) contiguos a alguno de distancia K−1:

  Esto se realiza iterativamente a medida que los estados se agregan a la cola con una distancia acumulada. El algoritmo BFS garantiza que todos los estados de distancia K se procesen antes que los de K+1.

- Se repite hasta que no se marque ninguno más
El bucle continúa mientras haya estados en la cola, lo que asegura que todos los estados alcanzables se procesen:
```
while queue:


- 2 SE USA ESA INFORMACIÓN PARA COMPARARLA CON EL TAMAÑO DE PATH PARA CADA ESTADO DE LA TABLA T.

Se usa la información de la distancia a la solución (steps_to_end) para compararla con el tamaño del camino (path_lengths) desde el estado inicial hasta cada estado en la tabla T. La comparación ocurre en la función compare_steps_and_paths.
```
def compare_steps_and_paths(steps_to_end, path_lengths):
    discrepancies = []
    for state, steps in steps_to_end.items():
        path_length = path_lengths.get(state, None)
        if path_length is not None and steps != path_length:
            discrepancies.append((state, steps, path_length))
    return discrepancies
```
Resultado si hay discrepancia:<br>
State: <estado>, Steps to End: <distancia>, Path Length: <longitud>


Si no las hay:<br>
No discrepancies found.
